In [9]:
!pip install selenium

In [10]:
!pip install webdriver-manager

In [58]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time 

In [51]:
# Create driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Get the page url 
page_url = "https://gilmoregirls.fandom.com/wiki/Category:Characters"
driver.get(page_url) # Open the page 

/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_18914/187490689.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [52]:
# Accept the cookies 
time.sleep(3) # Wait 3 seconds for the page to render 
driver.find_element(By.XPATH, "//div[text()='AKCEPTUJĘ']").click()

In [53]:
# Create a list of characters
character_element = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')


NameError: name 'pd' is not defined

In [54]:
characters = []
for value in character_element:
    characters_list = value.text
    characters.append({'name' : characters_list})

In [59]:
pd.DataFrame(characters)

,name
0,Category:Animals
1,Category:Chilton
2,Category:Family
3,Category:Guest characters
4,Category:Main Characters
...,...
157,Tristin Dugray
158,William Danes
159,Young Chui
160,Zach
